In [1]:
%pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [3]:
data_path_red_wine = "datasets/winequality-red.csv"
data = pd.read_csv(data_path_red_wine,sep=';')

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
41,8.8,0.61,0.30,2.8,0.088,17.0,46.0,0.99760,3.26,0.51,9.3,4
994,10.0,0.35,0.45,2.5,0.092,20.0,88.0,0.99918,3.15,0.43,9.4,5
1345,8.2,0.44,0.24,2.3,0.063,10.0,28.0,0.99613,3.25,0.53,10.2,6
1086,8.5,0.34,0.40,4.7,0.055,3.0,9.0,0.99738,3.38,0.66,11.6,7
1269,5.5,0.49,0.03,1.8,0.044,28.0,87.0,0.99080,3.50,0.82,14.0,8
1596,6.3,0.51,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
701,7.0,0.65,0.02,2.1,0.066,8.0,25.0,0.99720,3.47,0.67,9.5,6
1451,7.8,0.32,0.44,2.7,0.104,8.0,17.0,0.99732,3.33,0.78,11.0,7
568,9.8,0.50,0.49,2.6,0.250,5.0,20.0,0.99900,3.31,0.79,10.7,6
1265,7.2,0.57,0.05,2.3,0.081,16.0,36.0,0.99564,3.38,0.60,10.3,6


In [4]:
remote_server_uri = "http://0.0.0.0:5001"
mlflow.set_tracking_uri(remote_server_uri)

In [5]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5001'

In [8]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2025/06/25 11:22:00 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1750830720918, experiment_id='1', last_update_time=1750830720918, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

**ConnectionRefusedError**:  
It appears that the notebook cannot connect to the MLflow tracking server at `http://0.0.0.0:5001`.  
- Make sure the MLflow server is running and accessible at this address.
- You can start a local MLflow server with:  
  ```
  mlflow server --host 0.0.0.0 --port 5001 --backend-store-uri sqlite:///mlflow.db
  ```
- If running remotely, ensure network/firewall settings allow access.

In [9]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)

    return rmse, mae, r2

def load_data(data_path):
    data = pd.read_csv(data_path,sep=';')

    #split
    train, test = train_test_split(data)

    train_X = train.drop(["quality"], axis=1)
    test_X = test.drop(['quality'], axis=1)
    train_y = train[['quality']]
    test_y = test[['quality']]

    return train_X, train_y, test_X, test_y

def train(alpha=0.5, l1_ratio=0.5):
    warnings.filterwarnings('ignore')
    np.random.seed(40)

    train_X, train_y, test_X, test_y = load_data(data_path_red_wine)

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_X, train_y)

        # eval metric
        predict_qualities = lr.predict(test_X)
        rmse, mae, r2 = eval_metrics(test_y, predict_qualities)

        print("ElasticNet Model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("RMSE: %s" %rmse)
        print("MAE: %s" %mae)
        print("R2: %s" %r2)

        # log params
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path_red_wine)

        print("Save to: {}".format(mlflow.get_artifact_uri()))

        mlflow.sklearn.log_model(lr, 'model')

In [10]:
train(0.5, 0.5)

ElasticNet Model (alpha=0.500000, l1_ratio=0.500000):
RMSE: 0.793164022927685
MAE: 0.6271946374319586
R2: 0.10862644997792636


2025/06/25 11:23:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Save to: mlflow-artifacts:/1/686ffbb856a3426eb5656bebbe84e1c2/artifacts


2025/06/25 11:23:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run bold-owl-191 at: http://0.0.0.0:5001/#/experiments/1/runs/686ffbb856a3426eb5656bebbe84e1c2
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/1


In [11]:
train(0.2, 0.2)

2025/06/25 11:24:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


ElasticNet Model (alpha=0.200000, l1_ratio=0.200000):
RMSE: 0.7336400911821402
MAE: 0.5643841279275427
R2: 0.2373946606358417
Save to: mlflow-artifacts:/1/e48e027f681243089156a95e79a1f54e/artifacts


2025/06/25 11:24:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run abrasive-whale-993 at: http://0.0.0.0:5001/#/experiments/1/runs/e48e027f681243089156a95e79a1f54e
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/1
